# Convolutional Neural Network

### Importing the libraries

In [ ]:
!pip install --upgrade tensorflow keras Pillow

  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.3.2
    Uninstalling Pillow-8.3.2:
      Successfully uninstalled Pillow-8.3.2


You should consider upgrading via the 'c:\users\samue\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [1]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='tanh'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 79s 313ms/step - loss: 0.6724 - accuracy: 0.5730 - val_loss: 0.6926 - val_accuracy: 0.6180
Epoch 2/25
250/250 [==============================] - 35s 141ms/step - loss: 0.5979 - accuracy: 0.6754 - val_loss: 0.5831 - val_accuracy: 0.6945
Epoch 3/25
250/250 [==============================] - 34s 135ms/step - loss: 0.5440 - accuracy: 0.7232 - val_loss: 0.5378 - val_accuracy: 0.7350
Epoch 4/25
250/250 [==============================] - 35s 141ms/step - loss: 0.5018 - accuracy: 0.7506 - val_loss: 0.5082 - val_accuracy: 0.7645
Epoch 5/25
250/250 [==============================] - 33s 132ms/step - loss: 0.4854 - accuracy: 0.7694 - val_loss: 0.4791 - val_accuracy: 0.7740
Epoch 6/25
250/250 [==============================] - 34s 136ms/step - loss: 0.4626 - accuracy: 0.7850 - val_loss: 0.4578 - val_accuracy: 0.7900
Epoch 7/25
250/250 [==============================] - 34s 137ms/step - loss: 0.4408 - accuracy: 0.7900 - val_loss: 0.4739 - val_ac

## Part 4 - Making a single prediction

In [38]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [39]:
print(prediction)

cat


## Fim